In [1]:
## Perform Mean-Squared model testing and export results to review 

In [2]:
import pandas as pd

import glob

from pathlib import Path
import csv

In [3]:
# Initial imports
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import model_from_json
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [4]:
## Load pickle for exports and imports of data  
import pickle 
def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
    
def save_obj(obj, path ):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [5]:
path = Path('../Resources/finished_data_dict.pkl')
data_dict_import = load_obj(path)
path = Path('../Resources/finished_key_list.csv')
key_list_import_df = pd.read_csv(path,index_col=0)
key_list = []
for i in key_list_import_df['Symbol']:
    key_list.append(i)
len(key_list)

9992

In [6]:
ftd_key_list = []
etf_key_list = []
eqt_key_list = []
float_count_is_0 = 0

for i in key_list:
    index_value = i
    float_ftd_pct_ytd = data_dict_import[index_value]['ftd_stats']['float_ftd_pct_ytd']
    ## Check if there's float data. Otherwise, calculate using Outstanding share data
    ## ETFs have no float data, so they always use Outstanding Shares 
    if float_ftd_pct_ytd == 0:        
        os_ftd_pct_ytd = data_dict_import[index_value]['ftd_stats']['os_ftd_pct_ytd']
        if os_ftd_pct_ytd >= 5:
            float_count_is_0 += 1
            ftd_key_list.append(i)
            isEtf = data_dict_import[index_value]['companyProfile']['isEtf']
            if isEtf == True:
                etf_key_list.append(i)
            else:
                eqt_key_list.append(i)
    else:
        if float_ftd_pct_ytd >= 5:
            ftd_key_list.append(i)
            isEtf = data_dict_import[index_value]['companyProfile']['isEtf']
            if isEtf == 'True':
                etf_key_list.append(i)
            else:
                eqt_key_list.append(i)

print(f'Float = 0 for '+str(float_count_is_0)+' symbols')
print('All list length:',len(ftd_key_list))
print('ETF list length:',len(etf_key_list))
print('EQT list length:',len(eqt_key_list))

Float = 0 for 2112 symbols
All list length: 3695
ETF list length: 1872
EQT list length: 1823


In [7]:
nn = Sequential()

In [8]:
## Make a function to run this kind of model 

def create_mean_squared_model(symbol,validation_split=0.3,epochs=800,layers=2):
#     index_value = 0
#     for i in range(len(key_list)):
#         if key_list[i] == symbol:
#             index_value = i
#             pass
#     dict_key = key_list[index_value]
    df = data_dict_import[symbol]['dataFrame'].copy()
    X = df.drop(columns={'close','adjClose'}).values
    y = df['close'].values
    
    scaler = StandardScaler().fit(X)
    X = scaler.transform(X)    
    
    num_of_inputs = 16
    num_of_outputs= 1
        
    if layers == 2:
        # Define the model - deep neural network with two layers
        nn2 = Sequential()

        # First hidden layer
        nn2.add(Dense(units=8, input_dim=num_of_inputs, activation="relu"))

        # Second hidden layer
        nn2.add(Dense(units=8, activation="relu"))

        # Output layer
        nn2.add(Dense(units=num_of_outputs, activation="linear"))
        
        # Compile the model
        nn2.compile(loss="mean_squared_error", optimizer="adam", metrics=["mse"])

        # Fit the model
        model_2 = nn2.fit(X, y, validation_split=0.3, epochs=800, verbose=0)
        
        # Save model_2 as JSON
        nn_json2 = nn2.to_json()

        file_path = Path('../Model_Data/'+symbol+'model_2Layers.json')
        with open(file_path, "w") as json_file:
            json_file.write(nn_json2)

        # Save weights
        file_path = Path('../Model_Data/'+symbol+'model_2Layers_weights.h5')
        nn2.save_weights(file_path)
        
#         # load json and create model
#         file_path = Path('../Model_Data/'+symbol+'model_2Layers.json')
#         with open(file_path, "r") as json_file:
#             model_json = json_file.read()
#         loaded_model2 = model_from_json(model_json)

#         # load weights into new model
#         file_path = "'../Model_Data/'+symbol+'model_2Layers_weights.h5'"
#         loaded_model2.load_weights(file_path)
        
#         # Make some predictions with the loaded model
#         df2 = df.copy()
#         df2["predicted"] = loaded_model2.predict(X)
#         df_pred = df2[['close','predicted']]
#         df_pred
        
#     return df_pred

In [9]:
## Predict model based off exported model data 
def predict_model(symbol):
    ## Create data for prediction off data_dict
    df = data_dict_import[symbol]['dataFrame'].copy()
    X = df.drop(columns={'close','adjClose'}).values
    y = df['close'].values

    scaler = StandardScaler().fit(X)
    X = scaler.transform(X)      

    # load json and create model
    file_path = Path('../Model_Data/'+symbol+'model_2Layers.json')
    with open(file_path, "r") as json_file:
        model_json = json_file.read()
    loaded_model2 = model_from_json(model_json)

    # load weights into new model
    file_path = Path('../Model_Data/'+symbol+'model_2Layers_weights.h5')
    loaded_model2.load_weights(file_path)

    # Make some predictions with the loaded model
    df2 = df.copy()
    df2["predicted"] = loaded_model2.predict(X)
    df_pred = df2[['close','predicted']]
    df_pred
    
    return df_pred

In [10]:
test_symbol = 'GME'
create_mean_squared_model(test_symbol)
test_pred = predict_model(test_symbol)
test_pred

,close,predicted
Date,,
2016-01-04,28.309999,28.396738
2016-01-05,28.770000,28.668262
2016-01-06,28.370001,28.210463
2016-01-07,28.450001,28.422684
2016-01-08,28.370001,28.265915
...,...,...
2021-10-25,173.970000,165.223419
2021-10-26,177.840000,170.849884
2021-10-27,173.510000,164.030258


In [11]:
test_pred.tail(15)

,close,predicted
Date,,
2021-10-11,178.10,172.113159
2021-10-12,175.82,165.821625
2021-10-13,184.06,175.341843
2021-10-14,183.83,175.647400
2021-10-15,183.28,173.778488
2021-10-18,186.02,177.293488
2021-10-19,186.79,176.369720
2021-10-20,184.52,173.272385
2021-10-21,181.71,172.417130


In [12]:
## Need to figure out how to read finished model values better. 
## Can then re-run model multiple times and save the one with the best accuracy. 

In [ ]:
class NueralNetwork:
    
    def __init__(self):
        
        